In [1]:
# 1. Import Libraries (as you have it)
import torch
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
from torch.utils.tensorboard import SummaryWriter
import torchvision.transforms as T
import os

from DatasetCH import UpscaleDataset
import Network
from TrainDiffusion import EDMLoss, training_step, sample_model  # Import diffusion functions
from TrainUnet import train_step  # Import standard U-Net training

In [2]:

# 2. Define Datasets (as you have it)
ifs_dir = '/s2s/mpyrina/ECMWF_MCH/Europe_eval/s2s_hind_2022/all/'
obs_dir = '/net/cfc/s2s_nobackup/mpyrina/TABSD_ifs_like/'

dataset_train = UpscaleDataset(coarse_data_dir=ifs_dir, highres_data_dir=obs_dir,
                              year_start=2005, year_end=2008, month=815)
dataset_test = UpscaleDataset(coarse_data_dir=ifs_dir, highres_data_dir=obs_dir,
                             year_start=2009, year_end=2010, month=815)

dataloader_train = torch.utils.data.DataLoader(dataset_train, batch_size=8, shuffle=True, num_workers=4)
dataloader_test = torch.utils.data.DataLoader(dataset_test, batch_size=8, shuffle=False, num_workers=4)



# 4. Train U-Net with Diffusion
diffusion_model = Network.UNet(in_channels=2, out_channels=1, img_resolution=dataset_train.out_shape[0]).cuda() # Adjust in_channels if needed
diffusion_optimizer = torch.optim.AdamW(diffusion_model.parameters(), lr=1e-4)
diffusion_scaler = torch.cuda.amp.GradScaler()  # or torch.amp.GradScaler('cuda') if necessary
diffusion_loss_fn = EDMLoss()

num_epochs_diffusion = 50  # Adjust as needed
for epoch in range(num_epochs_diffusion):
    train_loss_diffusion = training_step(diffusion_model, diffusion_loss_fn,
                                         dataloader_train, diffusion_optimizer,
                                         diffusion_scaler, epoch, accum=4, device="cuda")
    print(f"Diffusion U-Net Epoch {epoch}: Loss = {train_loss_diffusion}")

# 5. Upscaling with Trained Diffusion Model
#   (This part needs careful implementation, using sample_model or a similar generation loop)
#   You'll need to generate from noise, conditioned on the low-resolution input.
#   This is not a single forward pass, but an iterative denoising process.
#   Use the sample_model function or adapt it.

Test - new upscale
Loaded coarse data shape: (138, 11, 11, 18)
Loaded high-resolution data shape: (138, 103, 241)
Final coarse shape: torch.Size([1518, 1, 11, 18])
Final fine shape: torch.Size([1518, 1, 103, 241])
Input shape (should be [N, 1, H, W]): torch.Size([1518, 1, 103, 241])
Dataset ready.
Test - new upscale
Loaded coarse data shape: (46, 11, 11, 18)
Loaded high-resolution data shape: (46, 103, 241)
Final coarse shape: torch.Size([506, 1, 11, 18])
Final fine shape: torch.Size([506, 1, 103, 241])
Input shape (should be [N, 1, H, W]): torch.Size([506, 1, 103, 241])
Dataset ready.


TypeError: 'int' object is not subscriptable

In [3]:
# 3. Train U-Net without Diffusion
unet_model = Network.UNet(in_channels=1, out_channels=1, img_resolution=dataset_train.out_shape[0]).cuda() # Adjust in_channels if needed
unet_optimizer = torch.optim.AdamW(unet_model.parameters(), lr=1e-4)
unet_scaler = torch.cuda.amp.GradScaler()  # or torch.amp.GradScaler('cuda') if necessary
unet_loss_fn = torch.nn.MSELoss()

num_epochs_unet = 10  # Adjust as needed
for epoch in range(num_epochs_unet):
    train_loss = train_step(unet_model, unet_loss_fn, dataloader_train,
                           unet_optimizer, unet_scaler, epoch, accum=4, device="cuda")
    print(f"U-Net (No Diffusion) Epoch {epoch}: Loss = {train_loss}")

TypeError: 'int' object is not subscriptable